In [22]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from Configurations import PneumoniaConfig

# Root directory of the project
ROOT_DIR = os.path.abspath("./")
MASK_RCNN = os.path.join(ROOT_DIR, 'Mask_RCNN')

# Import Mask RCNN
sys.path.append(MASK_RCNN)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

CONFIGURATIONS

In [ ]:
CONFIG = PneumoniaConfig()

#show configuration so far
CONFIG.display()

Dataset
Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, load_shapes(), and override the following methods:

load_image()

load_mask()

image_reference()

In [ ]:
class PneumoniaDataset(utils.Dataset):
    """
    Dataset class to help train our pneumonia dataset
    """

    def load_images(self,image_height, image_width):
        """Generate the requested number of synthetic images.
        height, width: the size of the generated images.
        """

        #add classes
        self.add_class('pneumonia', 1, 'Opacity Of Lungs')

        #add images
        

In [3]:
#Define Directories for train, test & Validation Set
train_dicom_images = os.path.join('Dataset','stage_2_train_images')
test_dicom_images = os.path.join('Dataset','stage_2_test_images')

In [4]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations




In [5]:
#handling training data
train_raw_data = pd.read_csv(os.path.join('/home/pirate/Documents/machine_learning/Pneumonia detection/Dataset', 'stage_2_train_labels.csv'))
train_raw_data.head(5)

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [9]:
train_image_files, train_image_details = parse_dataset(train_dicom_images, anns=train_raw_data)


In [10]:
ds = pydicom.read_file(train_image_files[0]) # read dicom image from filepath 
image = ds.pixel_array # get image array
image

array([[ 13,  37,  31, ...,  34,  40,  15],
       [ 44,  69,  65, ...,  69,  73,  47],
       [ 37,  64,  61, ...,  66,  67,  38],
       ...,
       [141, 181,  99, ...,  65,  76,  58],
       [138, 242, 205, ...,  78, 152, 129],
       [ 11,  14,  24, ...,   9,  26,   9]], dtype=uint8)